# Introduction

In [ ]:
import os

from openai import OpenAI

client = OpenAI(
    api_key=os.environ.get("OPENAI_API_KEY"),
)

openai_response = client.responses.create(
    model="gpt-4o-mini-2024-07-18",
    instructions="Extract name, years of experience, and primary skill from the job applicant description.",
    input="Khuyen Tran is a data scientist with 5 years of experience, skilled in Python and machine learning.",
)

print(openai_response.output_text)

In [ ]:
# Core Workflow

In [ ]:
import nest_asyncio

nest_asyncio.apply()

In [ ]:
from typing import List

from pydantic import BaseModel
from pydantic_ai import Agent


In [ ]:
class ApplicantProfile(BaseModel):
    first_name: str
    last_name: str
    experience_years: int
    primary_skill: List[str]


In [ ]:
agent = Agent(
    "gpt-4o-mini-2024-07-18",
    system_prompt="Extract name, years of experience, and primary skill from the job applicant description.",
    output_type=ApplicantProfile,
)

result = agent.run_sync(
    "Khuyen Tran is a data scientist with 5 years of experience, skilled in Python and machine learning."
)
print(result.output)

In [ ]:
result.output.model_dump()

In [ ]:
import pandas as pd

df = pd.DataFrame(result.output.model_dump())
df

# Using the DuckDuckGo Search Tool

In [ ]:
class UnemploymentDataSource(BaseModel):
    title: List[str]
    description: List[str]
    url: List[str]


In [ ]:
from pydantic_ai.common_tools.duckduckgo import duckduckgo_search_tool

# Define the agent with DuckDuckGo search tool
search_agent = Agent(
    "gpt-4o-mini-2024-07-18",
    tools=[duckduckgo_search_tool()],
    output_type=UnemploymentDataSource,
)

# Run a search for unemployment rate dataset
unemployment_result = search_agent.run_sync(
    "Monthly unemployment rate dataset for US from 2018 to 2024"
)

print(unemployment_result.output)

In [ ]:
unemployment_df = pd.DataFrame(unemployment_result.output.model_dump())
unemployment_df

## Comparison with LangChain Structured Output

In [ ]:
from typing import Optional


class RecipeExtractor(BaseModel):
    ingredients: List[str]
    instructions: str
    cook_time: Optional[str]


In [ ]:
recipe_agent = Agent(
    "gpt-4o-mini-2024-07-18",
    system_prompt="Pull ingredients, instructions, and cook time.",
    output_type=RecipeExtractor,
)

recipe_result = recipe_agent.run_sync(
    "Sugar, flour, cocoa, eggs, and milk. Mix, bake at 350F for 30 min."
)
print(recipe_result.output)
print(recipe_result.output.cook_time)

In [ ]:
from langchain_core.messages import HumanMessage, SystemMessage
from langchain_openai import ChatOpenAI

# Initialize the chat model
model = ChatOpenAI(model="gpt-4o-mini-2024-07-18", temperature=0)

# Bind the response formatter schema
model_with_tools = model.bind_tools([RecipeExtractor])

# Create a list of messages to send to the model
messages = [
    SystemMessage("Pull ingredients, instructions, and cook time."),
    HumanMessage(
        "Sugar, flour, cocoa, eggs, and milk. Mix, bake at 350F for 30 min."
    ),
]

# Invoke the model with the prepared messages
ai_msg = model_with_tools.invoke(messages)

# Access the tool calls made during the model invocation
print(ai_msg.tool_calls[0])
print(ai_msg.tool_calls[0]["args"]["cook_time"])
